In [2]:
import pandas as pd
import numpy as np

In [4]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
import tkinter.messagebox as mbox

# 파일 열기
def open_pd(addr):
    excel_df=pd.read_excel(addr, engine="openpyxl")
    excel_drop_df= excel_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)


    #0.0분 이수 데이터 삭제
    excel_drop_df=excel_drop_df[~excel_drop_df['강의 이수기간(분)'].isin([0.0])]
    start_date=int(input("단위기간 시작일(점 빼고 입력, ex/ 20221026 ) >> "))
    # 2단위기간 이전 데이터 삭제
    target = excel_drop_df['학습기준일'].ge(start_date)
    excel_drop_df=excel_drop_df[target]

    # 단위기간 최종 출결 
    excel_check_df = excel_drop_df.groupby(['차시구분','훈련생 성명']).max()
    print('<< 차시구분, 훈련생 성명 max >>')
    print(excel_check_df.head(3))

    excel_check_df = excel_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
    excel_check_df = excel_check_df.sort_values(by=['훈련생 성명', '학습기준일'])
    excel_check_df['이수시간(초)']=(excel_check_df['진도율(%)']*36).round(0)
    excel_check_df['이수시간(초)'].astype(int)

    excel_check_df= excel_check_df.drop(['진도율(%)','강의 이수기간(분)', '강의 시간(초)'], axis=1)


    print('<< drop 이후 데이터 >>')
    print(excel_check_df.head(3))

    # 파일 저장
    process_name = input("과정 명 >> ")
    excel_check_df.to_excel(r'C:\Users\Administrator\Downloads\(result)온라인출석 '+process_name+'.xlsx')





In [6]:
root = Tk()

def file_find():
    file = filedialog.askopenfilename(initialdir=r'C:/', title='select file', filetypes=(('excel file','*.xlsx'),('all files','*.*')))
    root.filename=file


def file_upload():
    if root.filename=='':
        mbox.showinfo('warning', 'Please selecct file')
        return
    else:
        open_pd(root.filename)
        root.destroy()
        # 여기서 file 불러와서 open_pd 함수 집어넣쟈
        return

root.title("출결 데이터 정제 프로그램")
root.geometry("500x200+200+200")
root.resizable(True, True)



try:   
    btn_find = tk.Button(root, text="find", width=10, command=file_find)
    btn_find.pack(side="right", padx=1, pady=1)
    btn_upload = tk.Button(root, text="산출하기", width=10, command=file_upload)
    btn_upload.pack(side="right", padx=1, pady=1)


except:
    print('error')
    root.destroy()


root.mainloop()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13084\652008093.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  excel_check_df = excel_check_df.groupby(['학습기준일', '훈련생 성명']).sum()


<< 차시구분, 훈련생 성명 max >>
                    훈련기관회원 PK       학습기준일          스타트        스타트타임  \
차시구분       훈련생 성명                                                     
IoT와AI_1차시 고미연     2019110001  20221125.0  2022.11.25   오전 10:49:28   
           김규림     2019112006  20221125.0  2022.11.25    오후 2:50:41   
           김민지     2020160007  20221125.0  2022.11.25   오전 12:25:50   

                                   학습시작시간                  학습종료시간  강의 이수기간(분)  \
차시구분       훈련생 성명                                                               
IoT와AI_1차시 고미연     2022.11.25 오전 10:49:28  2022.11.25 오전 11:19:32        25.0   
           김규림      2022.11.25 오후 2:50:41   2022.11.25 오후 3:16:08        25.0   
           김민지     2022.11.25 오전 12:25:50   2022.11.25 오전 2:20:39        25.0   

                   강의 시간(초)  진도율(%)  
차시구분       훈련생 성명                    
IoT와AI_1차시 고미연       3600.0   100.0  
           김규림       3600.0   100.0  
           김민지       3600.0   100.0  
<< drop 이후 데이터 >>
       

In [ ]:
def file_find():
    file = filedialog.askopenfilename(initialdir=r'C:/', title='select file', filetypes=(('excel file','*.xlsx'),('all files','*.*')))
    en_filepath.delete(0, END)
    en_filepath.insert(END, file[0])

def file_upload():
    if len(en_filepath.get())==0:
        mbox.showinfo('warning', 'Please selecct file')
        return
    else:
        open_pd(en_filepath.get())
        en_filepath.delete(0,END)
        # 여기서 file 불러와서 open_pd 함수 집어넣쟈
        return


root = Tk()
root.title("출결 데이터 정제 프로그램")
root.geometry("500x200+200+200")
root.resizable(True, True)

en_filepath = Entry(root, width=100)
en_filepath.pack(fill='x', padx =1, pady=1)

fr_bt=Frame(root)
fr_bt.pack(fill='x', padx=1, pady=1)


try:   
    # def open_addr():
        # root.filename = filedialog.askopenfilename(initialdir=r'C:/', title='select file', filetypes=(('excel file','*.xlsx'),('all files','*.*')))
        # if root.filename == '' : exit()
    btn_find = Button(root, text="find", width=10, command=file_find())
    btn_find.pack(side="right", padx=1, pady=1)
    
    # Label(root, text=root.filename).pack()

    btn_upload = Button(root, text="upload", width=10, command=file_upload())
    btn_upload.pack(side="right", padx=1, pady=1)


except:
    print('error')
    root.destroy()

# def open_addr():
#     root.filename = filedialog.askopenfilename(initialdir=r'C:/', title='select file', filetypes=(('excel file','*.xlsx'),('all files','*.*')))
#     if root.filename == '' : exit()
# btn_find = Button(root, text="find", width=10, command=open_addr())
# btn_find.pack(side="right", padx=1, pady=1)
# btn_upload = Button(root, text="upload", width=10, command=open_pd(root.filename))
# btn_upload.pack(side="right", padx=1, pady=1)
# Label(root, text=root.filename).pack()


root.mainloop()

In [ ]:
ai_df = pd.read_excel(r'C:\Users\Administrator\Desktop\error\code\데이터셋 수정 - AI제조.xlsx')
ai_drop_df = ai_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)
print(ai_drop_df.head(3))
ai_drop_df = ai_drop_df.drop(0, axis=0)

iot_df = pd.read_excel(r'C:\Users\Administrator\Desktop\error\code\데이터셋 수정 - IOT 가전 스마트홈.xlsx')
iot_drop_df = iot_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)
print(iot_drop_df.head(3))
iot_drop_df = iot_drop_df.drop(0, axis=0)

semi_df = pd.read_excel(r'C:\Users\Administrator\Desktop\error\code\데이터셋 수정 - 지능형반도체.xlsx')
semi_drop_df = semi_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)
print(semi_drop_df.head(3))
semi_drop_df = semi_drop_df.drop(0, axis=0)

In [ ]:
#0.0분 이수 데이터 삭제
ai_df=ai_drop_df[~ai_drop_df['강의 이수기간(분)'].isin([0.0])]
iot_df=iot_drop_df[~iot_drop_df['강의 이수기간(분)'].isin([0.0])]
semi_drop_df=semi_drop_df[~semi_drop_df['강의 이수기간(분)'].isin([0.0])]

# 2단위기간 이전 데이터 삭제
target_ai = ai_drop_df['학습기준일'].ge(20221026)
ai_drop_df=ai_drop_df[target_ai]

target_iot = iot_drop_df['학습기준일'].ge(20221026)
iot_drop_df=iot_drop_df[target_iot]

target = semi_drop_df['학습기준일'].ge(20221026)
semi_drop_df=semi_drop_df[target]

In [ ]:
# 단위기간 최종 출결 
ai_check_df = ai_drop_df.groupby(['차시구분','훈련생 성명']).max()

ai_check_df = ai_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
ai_check_df = ai_check_df.sort_values(by=['훈련생 성명', '학습기준일'])

ai_check_df['이수시간(초)']=(ai_check_df['진도율(%)']*36).round(0)
ai_check_df['이수시간(초)'].astype(int)

ai_check_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)온라인출석 AI.xlsx')

In [ ]:
# 단위기간 최종 출결 
iot_check_df = iot_drop_df.groupby(['차시구분','훈련생 성명']).max()

iot_check_df = iot_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
iot_check_df = iot_check_df.sort_values(by=['훈련생 성명', '학습기준일'])

iot_check_df['이수시간(초)']=(iot_check_df['진도율(%)']*36).round(0)
iot_check_df['이수시간(초)'].astype(int)

iot_check_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)온라인출석 iot.xlsx')

In [ ]:
# 단위기간 최종 출결 
semi_check_df = semi_drop_df.groupby(['차시구분','훈련생 성명']).max()

semi_check_df = semi_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
semi_check_df = semi_check_df.sort_values(by=['훈련생 성명', '학습기준일'])

semi_check_df['이수시간(초)']=(semi_check_df['진도율(%)']*36).round(0)
semi_check_df['이수시간(초)'].astype(int)

semi_check_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)온라인출석 반도체.xlsx')

In [ ]:
# 출결이벤트 대상자 찾기

# 학습시작시간에서 점 삭제
ai_drop_df.replace(to_replace=r'[.]',value="", regex=True, inplace=True)

# # 학습 시작일, 시작시간 구분
target_date_ai=ai_drop_df['학습시작시간'].str.slice(0, 8) 
ai_drop_df.insert(4,'시작일', target_date_ai)
ai_perfact_df=ai_drop_df

ai_perfact_df = ai_perfact_df.groupby(['훈련생 성명', '시작일', '차시구분']).max()
ai_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(test-1 1)퍼펙트출석 ai.xlsx')


ai_perfact_df = ai_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()

ai_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(test-1 2)퍼펙트출석 ai.xlsx')

In [ ]:
# 출결이벤트 대상자 찾기

# 학습시작시간에서 점 삭제
iot_drop_df.replace(to_replace=r'[.]',value="", regex=True, inplace=True)

# 학습 시작일, 시작시간 구분
target_date_iot=iot_drop_df['학습시작시간'].str.slice(0, 8) 
iot_drop_df.insert(4,'시작일', target_date_iot)
iot_perfact_df=iot_drop_df

iot_perfact_df = iot_perfact_df.groupby(['훈련생 성명', '시작일','차시구분']).max()
iot_perfact_df = iot_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()

iot_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)퍼펙트출석 iot.xlsx')

In [ ]:
# 출결이벤트 대상자 찾기

# 학습시작시간에서 점 삭제
semi_drop_df.replace(to_replace=r'[.]',value="", regex=True, inplace=True)

# 학습 시작일, 시작시간 구분
target_date_semi=semi_drop_df['학습시작시간'].str.slice(0, 8) 
semi_drop_df.insert(4,'시작일', target_date_semi)
semi_perfact_df=semi_drop_df

semi_perfact_df = semi_perfact_df.groupby(['훈련생 성명', '시작일','차시구분']).max()
semi_perfact_df = semi_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()

semi_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)퍼펙트출석 반도체.xlsx')